# Задача 5

### Случайная величина распределена равномерно на отрезке [θ, 2θ].
### a) По выборке объёма n найти оценки параметра θ методом моментов и методом максимального правдоподобия.
### b) Проверить оценки на несмещённость и состоятельность. Исправить эти оценки, если необходимо.
### c) Сравнить эффективность исправленных оценок.
### d) Построить точный доверительный интервал для параметра θ.
### e) Построить асимптотический доверительный интервал для параметра θ.

![](img/page_01.jpg)

![](img/page_02.jpg)

![](img/page_03.jpg)

![](img/page_04.jpg)

![](img/page_05.jpg)

![](img/page_06.jpg)

### f) Сгенеруйте выборку объёму n = 100 для некоторого значения парамeтра. Вычислите указанные выше доверительные интегралы для доверительной веротяности 0.95.

In [185]:
import math
import pandas as pd
import random
import numpy as np
from rich.console import Console
from rich.panel import Panel
from rich.text import Text

console = Console()

tetta = 5  # Любимая оценка)
betta = 0.95


def p(x: float) -> float:
    return 1 / tetta if (tetta <= x <= 2 * tetta) else 0


def F(x: float) -> float:
    if x < tetta:
        return 0
    elif tetta <= x <= 2 * tetta:
        return x / tetta - 1
    elif x > tetta:
        return 1


def inverse_F(y: float) -> float:
    return (y + 1) * tetta


n = 100

sampling = pd.Series(inverse_F(random.uniform(0, 1)) for _ in range(n))  # Выборка

### Посчитаем точный доверительный интервал

In [186]:
x_max = np.max(sampling)

t1 = ((1 - betta) / 2) ** (1 / n)
t2 = ((1 + betta) / 2) ** (1 / n)

lower_grade = x_max / (t2 + 1)
upper_grade = x_max / (t1 + 1)

console.print(f"Доверительный интервал: {lower_grade} < θ < {upper_grade}")
console.print(f"Длина интервала: {upper_grade - lower_grade}")

Доверительный интервал: 4.95994550377298 < θ < 5.0507789631729265

Длина интервала: 0.09083345939994647

### Посчитаем асимптотический доверительный интервал

In [187]:
x_mean = np.mean(sampling)
tetta_wave = (2 / 3) * x_mean

s = np.sqrt((np.sum([(x - x_mean) ** 2 for x in sampling])) / (n - 1))

t1 = -1.96
t2 = 1.96

lower_grade = tetta_wave - 2 * s * np.sqrt(n - 1) / (3 * n) * t2
upper_grade = tetta_wave - 2 * s * np.sqrt(n - 1) / (3 * n) * t1

console.print(f"Доверительный интервал: {lower_grade} < θ < {upper_grade}")
console.print(f"Длина интервала: {upper_grade - lower_grade}")

Доверительный интервал: 4.955232986130193 < θ < 5.34058694273968

Длина интервала: 0.385353956609487

### g) Численно постройте бутстраповский доверительный интервал.

In [188]:
# Оценка ОММ
h_wave_omm = tetta_wave

# Исправленная оценка ОММП
h_wave_ommp = (n + 1) * x_max / (2 * n + 1)

bootstrap_deltas_omm = []

for _ in range(1000):
    subsampling = np.random.choice(sampling, size=n)
    bootstrap_deltas_omm.append((2 / 3) * np.mean(subsampling) - h_wave_omm)

variation_row_omm = sorted(bootstrap_deltas_omm)


bootstrap_deltas_ommp = []

for _ in range(1000):
    subsampling = np.random.choice(sampling, size=n)
    bootstrap_deltas_ommp.append(
        (n + 1) * np.max(subsampling) / (2 * n + 1) - h_wave_ommp
    )

variation_row_ommp = sorted(bootstrap_deltas_ommp)

t1_omm = variation_row_omm[int((1 - betta) * 1000 / 2)]
t2_omm = variation_row_omm[int((1 + betta) * 1000 / 2)]

t1_ommp = variation_row_ommp[int((1 - betta) * 1000 / 2)]
t2_ommp = variation_row_ommp[int((1 + betta) * 1000 / 2)]

lower_grade_omm = h_wave_omm - t2_omm
upper_grade_omm = h_wave_omm - t1_omm

lower_grade_ommp = h_wave_ommp - t2_ommp
upper_grade_ommp = h_wave_ommp - t1_ommp

console.print(f"Бутстраповский доверительный интервал, полученный с помощью оценки параметра методом ОММ:\n {lower_grade_omm} < θ < {upper_grade_omm}")
console.print(f"Длина интервала: {upper_grade_omm - lower_grade_omm}")

console.print()

console.print(f"Бутстраповский доверительный интервал, полученный с помощью оценки параметра методом ОММП:\n {lower_grade_ommp} < θ < {upper_grade_ommp}")
console.print(f"Длина интервала: {upper_grade_ommp - lower_grade_ommp}")

Бутстраповский доверительный интервал, полученный с помощью оценки параметра методом ОММ:
 4.95580934333009 < θ < 5.330634448525707

Длина интервала: 0.37482510519561707

Бутстраповский доверительный интервал, полученный с помощью оценки параметра методом ОММП:
 4.98399093093934 < θ < 5.056710256699337

Длина интервала: 0.07271932575999696

### h) Сравнить все интервалы.
#### В итоге, мы получаем, что бутстраповский доверительный интервал < точный доверительный интервал < асимптотический доверительный интервал. Причём у бутстраповского доверительного интервала, полученного с помощью оценки параметра методом ОММП, длина меньше, чем у полученного с помощью ОММ.